In [ ]:
## Following code below utilizies USPS API to verify SBA loan application addresses.
## NOTE it took sagemaker ml.g5.16xlarge ~160 hours to verify 11.5 million addresses

In [2]:
import glob
import pandas as pd
import requests
pd.set_option('display.max_columns', None)
import time
from sagemaker import get_execution_role, Session
import xml.etree.ElementTree as ET 
import concurrent.futures
import requests
import xml.etree.ElementTree as ET
import time

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
# USPS API key
api_key_pass = '816560NHDO3688A'
api_key_id = '98UNIVE9Y7817'
# USPS API endpoint
endpoint = 'https://secure.shippingapis.com/ShippingAPI.dll'
output_file = 'verification_results_parallel_v2.txt'
# Function to verify an address with retries and write results to a file
def verify_address_with_retry_and_write(address, output_file='verification_results_parallel_v2.txt', max_retries=3):
    
    for retry in range(max_retries):
        try:
            params = {
                'API': 'Verify',
                'XML': f'''
                    <AddressValidateRequest USERID="{api_key_id}" PASSWORD ="{api_key_pass}">
                        <Address ID="0">
                            <FirmName>{address['BorrowerName']}</FirmName>
                            <Address1>{address['BorrowerAddress']}</Address1>
                            <Address2></Address2>
                            <City>{address['BorrowerCity']}</City>
                            <State></State>
                            <Zip5>{address['BorrowerZip']}</Zip5>
                            <Zip4></Zip4>
                        </Address>
                    </AddressValidateRequest>
                '''
            }

            # Send the API request
            response = requests.get(endpoint, params=params)

            # HTTP status code of a successful network response else return statement that the API request was not successful
            if response.status_code == 200:
                # Parse the XML response
                root = ET.fromstring(response.text)
                # Finds any error tags
                error = root.find('.//Error')
                # Finds a return text tag requesting more information for a valid address
                not_validated = root.find('.//ReturnText')

                # If there was an error in the response
                if error is not None:
                    time.sleep(5)
                    continue
                # If address entered was not validated
                elif not_validated is not None:
                    result = "More Information Needed to Validate Address"
                # Return true if address was validated
                else:
                    result = True
            else:
                print(f"HTTP Error: {response.status_code}. Retrying in 5 seconds...")
                time.sleep(5)
                continue
        except ConnectionError as e:
            print(f"ConnectionError: {e}. Retrying in 5 seconds...")
            time.sleep(5)
            continue

        # Write the result to the output file
        with open(output_file, 'a') as file:
            file.write(f"{address['LoanNumber']};{address['BorrowerName']};{address['BorrowerAddress']};{address['BorrowerCity']};{address['BorrowerZip']};{result}\n")

        return result

In [4]:
# Function to parallelize the address verification process
def parallelize_verification(address_data):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(verify_address_with_retry_and_write, row) for _, row in address_data.iterrows()]

        for future in concurrent.futures.as_completed(futures):
            result = future.result()


In [5]:
# Lists to store data
rows = []

# Read the file line by line
with open('verification_results_parallel.txt', 'r') as file:
    for line in file:
        # Split the line using semicolon as the delimiter
        fields = line.strip().split(';')
        
        # Handle lines with 6 or 7 fields
        if len(fields) == 6 or len(fields) == 7 or len(fields) == 8 or len(fields) == 9 or len(fields) == 10:
            # Append the fields to the rows list
            rows.append(fields)
        else:
            # Handle lines with a different number of fields (optional)
            print(f"Skipping line: {line}")

# Create a DataFrame from the collected rows
df = pd.DataFrame(rows, columns=['Field1', 'Field2', 'Field3', 'Field4', 'Field5', 'Field6', 'Field7','Field8', 'Field9', 'Field10'])

In [6]:
#SBA loan data set
bucket='sagemaker-studio-8kjlfkwa9aq'
data_key = 'ppp-data-combined.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

combined_data = pd.read_csv(data_location)

/tmp/ipykernel_22/1984545531.py:5: DtypeWarning: Columns (17,51) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_data = pd.read_csv(data_location)


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9028879 entries, 0 to 9028878
Data columns (total 10 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   Field1   object
 1   Field2   object
 2   Field3   object
 3   Field4   object
 4   Field5   object
 5   Field6   object
 6   Field7   object
 7   Field8   object
 8   Field9   object
 9   Field10  object
dtypes: object(10)
memory usage: 688.8+ MB


In [13]:
combined_data.head()

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,SBAGuarantyPercentage,InitialApprovalAmount,CurrentApprovalAmount,UndisbursedAmount,FranchiseName,ServicingLenderLocationID,ServicingLenderName,ServicingLenderAddress,ServicingLenderCity,ServicingLenderState,ServicingLenderZip,RuralUrbanIndicator,HubzoneIndicator,LMIIndicator,BusinessAgeDescription,ProjectCity,ProjectCountyName,ProjectState,ProjectZip,CD,JobsReported,NAICSCode,Race,Ethnicity,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate
0,0,2593527208,04/16/2020,1094.0,PPP,"LORI L. HOLDER, LCSW",504 MAIN ST Suite 472,LEWISTON,ID,83501-1806,02/24/2021,Paid in Full,24,100,500.0,500.0,0.0,NaN,59698.0,Umpqua Bank,"445 SE Main St, First Fl",ROSEBURG,OR,97470-4934,U,Y,N,Existing or more than 2 years old,LEWISTON,NEZ PERCE,ID,83501-1806,ID-01,1.0,621330.0,Unanswered,Unknown/NotStated,NaN,500.0,NaN,NaN,NaN,NaN,NaN,Sole Proprietorship,59698.0,Umpqua Bank,ROSEBURG,OR,Unanswered,Unanswered,NaN,503.92,02/12/2021
1,1,2741007306,04/29/2020,1087.0,PPP,THUNDER HILL FITNESS CENTER LLC,283 S. Middlefork Rd,GARDEN VALLEY,ID,83622,06/22/2021,Paid in Full,24,100,500.0,500.0,0.0,NaN,106925.0,Mountain America FCU,9800 S Monroe St,SANDY,UT,84070-4419,R,N,N,New Business or 2 years or less,GARDEN VALLEY,BOISE,ID,83622-0001,ID-01,2.0,713940.0,Unanswered,Unknown/NotStated,NaN,500.0,NaN,NaN,NaN,NaN,NaN,Limited Liability Partnership,106925.0,Mountain America FCU,SANDY,UT,Unanswered,Unanswered,NaN,505.25,05/25/2021
2,2,3342508906,04/28/2021,1087.0,PPP,KATRINA RAMIREZ,2877 N Bliss Dr,Idaho Falls,ID,83401-5873,11/11/2021,Paid in Full,60,100,500.0,500.0,0.0,NaN,317954.0,"Newtek Small Business Finance, Inc.",1981 Marcus Avenue,LAKE SUCCESS,NY,11042,U,N,N,Existing or more than 2 years old,Idaho Falls,BONNEVILLE,ID,83401-5873,ID-02,1.0,423940.0,Unanswered,Unknown/NotStated,1.0,496.0,NaN,NaN,NaN,NaN,NaN,Self-Employed Individuals,317954.0,"Newtek Small Business Finance, Inc.",LAKE SUCCESS,NY,Unanswered,Unanswered,NaN,502.43,10/27/2021
3,3,3379798208,08/04/2020,1087.0,PPP,JAMES PARIS SCHOOL OF MUSIC,3048 W SAN REMO ST,MERIDIAN,ID,83646-3214,02/03/2021,Paid in Full,60,100,500.0,500.0,0.0,NaN,96211.0,Idaho Central CU,4400 Central Way,CHUBBUCK,ID,83202-5096,U,N,N,New Business or 2 years or less,MERIDIAN,ADA,ID,83646-3214,ID-01,1.0,812990.0,White,Not Hispanic or Latino,NaN,500.0,NaN,NaN,NaN,NaN,NaN,Sole Proprietorship,96211.0,Idaho Central CU,CHUBBUCK,ID,Male Owned,Unanswered,NaN,502.16,01/12/2021
4,4,3753429003,05/20/2021,1087.0,PPP,ANGEL RICE,4088 S Riva Ridge Way,Boise,ID,83709-4448,03/22/2022,Paid in Full,60,100,500.0,500.0,0.0,NaN,188361.0,"Prestamos CDFI, LLC",1024 East Buckeye Road Suite 270,Phoenix,AZ,85034,U,N,N,Existing or more than 2 years old,Boise,ADA,ID,83709-4448,ID-01,1.0,492210.0,White,Not Hispanic or Latino,NaN,499.0,NaN,NaN,NaN,NaN,NaN,Independent Contractors,188361.0,"Prestamos CDFI, LLC",Phoenix,AZ,Female Owned,Non-Veteran,NaN,503.40,02/03/2022


In [16]:
#checks if any loans were missed in the USPS API verification
data_remaining = combined_data[~combined_data['LoanNumber'].isin(df['Field1'].astype('int64'))]

In [17]:
data_remaining.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2020621 entries, 13 to 11765551
Data columns (total 54 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Unnamed: 0                   int64  
 1   LoanNumber                   int64  
 2   DateApproved                 object 
 3   SBAOfficeCode                float64
 4   ProcessingMethod             object 
 5   BorrowerName                 object 
 6   BorrowerAddress              object 
 7   BorrowerCity                 object 
 8   BorrowerState                object 
 9   BorrowerZip                  object 
 10  LoanStatusDate               object 
 11  LoanStatus                   object 
 12  Term                         int64  
 13  SBAGuarantyPercentage        int64  
 14  InitialApprovalAmount        float64
 15  CurrentApprovalAmount        float64
 16  UndisbursedAmount            float64
 17  FranchiseName                object 
 18  ServicingLenderLocationID    float64
 19

In [ ]:
parallelize_verification(data_remaining)